In [1]:
import pandas as pd

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def getFirstAdduct(x):
    if ";" in x:
        adduct = str("[" + x.split(";")[1].split("^")[1].split("_")[0].strip() + "]+")
    else:
        adduct = str("[" + x.split("^")[1].split("_")[0].strip() + "]+")
    return adduct

In [2]:
# 02
#non_overlapping_p = "/Volumes/elements/MetaboDIA/Option_1/combined_library_02_67/prepare_lib/non_overlapping_mdia_67.txt"
#oms_lib = "/Volumes/elements/MetaboDIA/Option_1/combined_library_02_67/prepare_lib/20201129_s45_mt1_resolve_totoc_67_02.tsv"
#mdia_oms_lib = "/Volumes/elements/MetaboDIA/Option_1/combined_library_02_67/prepare_lib/20201205_DDA_tdlib_msconvert_mzXML_10ppm_min_s02_pf00.tsv"
#combined_out = "/Volumes/elements/MetaboDIA/Option_1/combined_library_02_67/prepare_lib/20201215_combined_mdiaoms_library_67.tsv"

# only 140 of the 204 non_overlapping compounds are found in the mdia_oms library, because 
# the other where filtered out in the conversion process, since the 
# compounds need to have a least 3 transitions for library generation

no = pd.read_csv(non_overlapping_p, sep = "\t")
oms = pd.read_csv(oms_lib, sep = "\t")
mdia = pd.read_csv(mdia_oms_lib, sep = "\t")

comp = []
for element in no['Compounds'].tolist():
    comp.append(element)
    comp.append(str(element + "_decoy"))

no = pd.DataFrame(comp, columns=['Compounds'])

## merge based on non_overlapping
merged = pd.merge(no, mdia, left_on='Compounds', right_on='CompoundName', how='inner')
merged = merged.drop(columns=['Compounds'])

merged['Adducts'] = merged['CompoundName'].apply(getFirstAdduct)
merged['Annotation'] = "NA"

frames = [oms,merged]
result = pd.concat(frames)

result.to_csv(combined_out, sep = '\t',index=False)

## There seem to be an issue with empty lines at the end of the export file.
## Please open the file with excel and save it again, then everything should 
## work in terms of conversion to pqp using the OpenMS::TargetedFileConverter